In [1]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Mindsay/mathieu/Benter-Project

/home/mathieu/Mindsay/mathieu/Benter-Project


In [2]:
from typing import Generator, Optional

import datetime as dt
import os
import json

from constants import UNIBET_MIN_DATE

In [ ]:
# Found in JS unibet code
l = { INITIAL: { statusCode: 0 }, UPCOMING: { statusText: "NOT_STARTED", statusCSS: "RACE_UPCOMING", statusMessage: "", statusCode: 1 }, STARTED: { statusText: "STARTED", statusCSS: "RACE_STARTED", statusMessage: "Les chevaux sont partis", statusCode: 2 }, FINISHED: { statusText: "FINISHED_OFFICIAL", statusCSS: "RACE_FINISHED", statusMessage: "Course terminée", statusMessageLong: "La course est terminée", statusCode: 6 }, PROVISIONAL: { statusText: "FINISHED_PROVISIONAL", statusCSS: "RACE_FINISHED", statusMessage: "Course terminée", statusMessageLong: "La course est terminée", inquiryStatus: 8, inquiryStatusBis: 10, statusCodeFrom: 7, statusCodeTo: 10 }, CANCELLED: { statusCSS: "RACE_CANCELLED", statusText: "CANCELLED", statusMessage: "Course anulée", statusCode: 11 } },
u = { meetings: "meetings", upcoming: "upcoming" },
d = { rates: "rates", results: "results", betrunners: "betrunners" },
p = { SIMPLE: { name: "Simple", index: "SIMPLE", categoryName: "S", css: "bet-simple", field: !1, minRunners: a[1], help: "Mise de base: 1€<br>Trouvez le 1er cheval à l’arrivée gagnant ou placé." }, JUMELE: { name: "Jumelé", index: "JUMELE", categoryName: "J", css: "bet-jumele", field: !0, minRunners: a[3], help: "Mise de base: 1€<br>Trouvez les 2 premiers chevaux à l’arrivée dans l’ordre ou le désordre." } },
f = { WINNER: "WINNER", PLACED: "PLACED", REDUIT: "REDUIT", TOTAL: "TOTAL" },
m = { WINNER_PLACED: "WINNER_PLACED", WINNER: "WINNER", PLACED: "PLACED", DEFAULT: "DEFAULT" },
h = { 1: { category: "SIMPLE", categoryName: "S<sup>G</sup>", formula: "WINNER", name: "Simple Gagnant", css: "bet-simple-gagnant", field: !1, help: "Mise de base: 1€<br>Trouvez le 1er cheval de l’arrivée.", type: 1 }, 2: { category: "SIMPLE", categoryName: "S<sup>P</sup>", formula: "PLACED", name: "Simple Placé", css: "bet-simple-place", field: !1, help: "Mise de base: 1€<br>Trouvez 1 cheval parmi les 3 premiers sur les courses de 8 chevaux et plus OU 1 cheval parmi les 2 premiers sur les courses de 4 à 7 chevaux.", type: 2 }, 3: { category: "JUMELE", categoryName: "J<sup>G</sup>", formula: "WINNER", name: "Jumelé Gagnant", css: "bet-jumele-gagnant", field: !0, help: "Mise de base: 1€<br>Trouvez les 2 premiers chevaux dans l’ordre ou le désordre.", type: 3 }, 4: { category: "JUMELE_ORDRE", categoryName: "J<sup>or</sup>", formula: "DEFAULT", name: "Jumelé Ordre", css: "bet-jumele-ordre", field: !0, order: !0, help: "Mise de base: 1€<br>Trouvez les 2 premiers chevaux dans l’ordre.", type: 4 }, 5: { category: "JUMELE", categoryName: "J<sup>P</sup>", formula: "PLACED", name: "Jumelé Placé", css: "bet-jumele-place", field: !0, help: "Mise de base: 1€<br>Trouvez 2 chevaux parmi les 3 premiers à l’arrivée.", type: 5 }, 6: { category: "TRIO", categoryName: "T", formula: "DEFAULT", name: "Trio", css: "bet-trio", field: !0, help: "Mise de base: 1€<br>Trouvez les 3 premiers chevaux.", type: 6 }, 7: { category: "LEBOULET", categoryName: "B", formula: "DEFAULT", name: "Le Boulet", css: "bet-boulet", field: !1, help: "Mise de base: 1€<br>Trouvez le cheval qui arrive à la 4ème place.", type: 7 }, 8: { category: "QUADRI", categoryName: "Q", formula: "DEFAULT", name: "Quadri", css: "bet-quadri", field: !0, miseMin: .5, help: "Mise de base: 0.50€<br>Trouvez les 4 premiers chevaux, dans l’ordre ou le désordre.", type: 8 }, 11: { category: "TRIO_ORDRE", categoryName: "T<sup>or</sup>", formula: "DEFAULT", name: "Trio Ordre", css: "bet-trio-ordre", field: !0, order: !0, help: "Mise de base: 1€<br>Trouvez les 3 premiers chevaux dans l'ordre.", type: 11 }, 12: { category: "5SUR5", categoryName: "5/5", formula: "DEFAULT", name: "5 sur 5", css: "bet-5-5", field: !0, miseMin: .5, help: "Mise de base: 0.50€<br>Trouvez les 5 premiers chevaux, dans l’ordre ou le désordre.", type: 12 }, 13: { category: "2SUR4", categoryName: "2/4", formula: "DEFAULT", name: "2 / 4", css: "bet-2-4", field: !0, help: "Mise de base: 1€<br>Trouvez 2 chevaux parmi les 4 premiers à l’arrivée.", type: 13 }, 29: { category: "DEUZIO", categoryName: "Dz", formula: "DEFAULT", name: "Le Deuzio", css: "bet-deuzio", help: "Mise de base: 1€<br>Trouvez le cheval qui arrive à la 2ème place.", type: 29 }, 15: { category: "MIX4", formula: "DEFAULT", name: "Mix 4", css: "bet-mix4", field: !0, miseMin: 1.5, help: "Mise de base : 1.50€<br>Mixez dans un même betslip: un Quadri + des Trios + des Jumelés Gagnants.", type: 15 }, 16: { category: "MIX5", formula: "DEFAULT", name: "Mix 5", css: "bet-mix5", field: !0, miseMin: 2, help: "Mise de base: 2€<br>Mixez dans un même betslip: un 5 sur 5 + des Quadris + des Trios.", type: 16 }, 31: { category: "MIXS", formula: "DEFAULT", name: "Mix S", css: "bet-mixs", miseMin: 3, help: "Mise de base: 3€<br>Mixez dans un même betslip: un Simple Gagnant + un Simple Placé + un Deuzio + un Boulet", type: 31 }, 32: { category: "JUMELE", categoryName: "J", formula: "WINNER_PLACED", name: "Jumelé GP", css: "bet-jumele-gagnantplace", field: !0, help: "-", type: 32 }, 33: { category: "SIMPLE", categoryName: "S", formula: "WINNER_PLACED", name: "Simple GP", css: "bet-simple-gagnantplace", help: "-", type: 33 } };

In [3]:
# to check with previous races: https://www.unibet.fr/zones/turf/statistiques.json?raceId=244793&runnerRank=7

class UnibetBetTRateType:
    simple_gagnant=1
    simple_place=2
    jumele_gagnant=3
    jumele_ordre=4
    jumele_place=5
    trio=6
    boulet=7
    quadri=8
    trio_ordre=11
    two_on_four = 13
    deuzio=29
    
    
class UnibetProbableType:
    morning_simple_gagnant_odds=5 #"matin cote" on simple_gagnant
    final_simple_gagnant_odds=6 # "cote directe" or "rapport_final" on simple_gagnant
    morning_simple_place_odds=7 # "matin cote" on simple_place
    final_simple_place_odds=8 # "cote directe" or "rapport_final" on simple_place
    unknown_probable_type3=9
    final_deuzio_odds=13 # "rapport_final" on deuzio

In [4]:
UNIBET_DATA_PATH="./data/Unibet"

In [5]:
def date_countdown_generator(start_date:dt.date, end_date:Optional[dt.date])->Generator[dt.date, None, None]:
    end_date=end_date or dt.date.today()
    current_date= start_date
    while current_date <=end_date:
        yield current_date
        current_date+=dt.timedelta(days=1)

In [6]:
seen_runner_id =set()
def process_race(race:dict, complete_race:dict)->None:
    if complete_race.get('note')=='server error, no json':
        # Can not use complete_race
        return None
    race_name=race['name']
    assert complete_race['name']==race['name']
    
    race_datetime=dt.datetime.fromtimestamp(complete_race['starttime']/1000)
    if race_datetime.year <2020:
        return None
        
    if complete_race['betCounter'] is not None:
        print(complete_race['betCounter'], complete_race['friendlyUrl'])
        
    pronostic = complete_race['details']['pronostic']
    

In [7]:
def process_horse_show(horse_show:dict)->None:
    horse_show_datetime=dt.datetime.fromtimestamp(horse_show['date']/1000)
    unibet_n_horse_show=horse_show['rank']
    horse_show_ground=horse_show['ground']
    horse_show_place = horse_show['place']
    horse_show_country = horse_show['country']
    


In [8]:
for date in date_countdown_generator(start_date=dt.date(2020,1,1), end_date=dt.date(2020,5,1)):
    day_folder_path=os.path.join(UNIBET_DATA_PATH, date.isoformat())
    if "programme.json" in os.listdir(day_folder_path):
        with open(os.path.join(day_folder_path, "programme.json"), 'r') as fp:
            programme=json.load(fp=fp)
        if 'data' not in programme:
            print(f'Can not import programme of {date.isoformat()}')
            continue
            
        horse_shows=programme['data']
        for horse_show in horse_shows:
            process_horse_show(horse_show=horse_show)
            
            if horse_show.get('races'):
                
                for race in horse_show['races']:
                    race_path = os.path.join(day_folder_path, f"R{horse_show['rank']}_C{race['rank']}.json")
                    with open(race_path, 'r') as fp:
                        complete_race=json.load(fp=fp)
                    process_race(race=race, complete_race=complete_race)

In [9]:
{runner['zeturfId'] for runner in complete_race['runners']}


{3078277,
 3078278,
 3078279,
 3078280,
 3078281,
 3078282,
 3078283,
 3078284,
 3078285,
 3078286,
 3078287}

In [10]:
complete_race.keys()

dict_keys(['get_url', 'query_date', 'zeturfId', 'meetingId', 'starttime', 'date', 'type', 'name', 'rank', 'conditions', 'stake', 'arjelLevel', 'distance', 'betCounter', 'friendlyUrl', 'runners', 'details', 'results'])

In [21]:
complete_race['friendlyUrl']

'/turf/race/01-05-2020-R7-C8-gulfstream-park-starter-optional-claiming.html'

In [12]:
complete_race['details'].keys()

dict_keys(['zeturfId', 'betList', 'pronostic', 'pronosticList', 'raceDuration', 'betState', 'photos', 'raceBetStatus', 'probables', 'rates'])

In [17]:
complete_race['details']['probables'].keys()

dict_keys(['5', '6', '7', '8', '9', '13'])

In [18]:
complete_race['details']['rates']

[{'type': '1', 'combination': '4', 'rate': '3.8'},
 {'type': '2', 'combination': '4', 'rate': '1.10'},
 {'type': '2', 'combination': '1', 'rate': '1.10'},
 {'type': '2', 'combination': '5', 'rate': '1.9'},
 {'type': '3', 'combination': '4-1', 'rate': '3.1'},
 {'type': '5', 'combination': '4-1', 'rate': '1.8'},
 {'type': '5', 'combination': '4-5', 'rate': '4.9'},
 {'type': '5', 'combination': '1-5', 'rate': '4.2'},
 {'type': '4', 'combination': '4-1', 'rate': '7.9'},
 {'type': '6', 'combination': '4-1-5', 'rate': '12.0'},
 {'type': '11', 'combination': '4-1-5', 'rate': '38.5'},
 {'type': '13', 'combination': '4-1', 'rate': '1.10'},
 {'type': '13', 'combination': '4-5', 'rate': '1.10'},
 {'type': '13', 'combination': '4-2', 'rate': '1.10'},
 {'type': '13', 'combination': '1-5', 'rate': '1.10'},
 {'type': '13', 'combination': '1-2', 'rate': '1.10'},
 {'type': '13', 'combination': '5-2', 'rate': '1.10'},
 {'type': '8', 'combination': '4-1-5-2', 'rate': '6.1'},
 {'type': '29', 'combination'

In [20]:
complete_race['details']['pronosticList']

[{'rank': 1,
  'number': 1,
  'name': 'DALARNA (US)',
  'comment': None,
  'long_comment': None},
 {'rank': 2,
  'number': 4,
  'name': 'HIGH NOON RIDER (US)',
  'comment': None,
  'long_comment': None},
 {'rank': 3,
  'number': 2,
  'name': 'FLOWMOTION (US)',
  'comment': None,
  'long_comment': None},
 {'rank': 4,
  'number': 10,
  'name': 'JAKARTA (US)',
  'comment': None,
  'long_comment': None},
 {'rank': 5,
  'number': 3,
  'name': 'CELEBRATION (GB)',
  'comment': None,
  'long_comment': None},
 {'rank': 6,
  'number': 7,
  'name': 'TIL THE END (US)',
  'comment': None,
  'long_comment': None}]

In [13]:
complete_race['details']['betList']

[{'type': 1,
  'rate': 86.5,
  'max': 450,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 2,
  'rate': 82.0,
  'max': 450,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 3,
  'rate': 75.0,
  'max': 50,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 5,
  'rate': 75.0,
  'max': 50,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 4,
  'rate': 74.0,
  'max': 50,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 6,
  'rate': 70.0,
  'max': 50,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 11,
  'rate': 70.0,
  'max': 50,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 13,
  'rate': 75.0,
  'max': 50,
  'open': False,
  'status': 16,
  'isolated': 0,
  'betstatus': 'PAID'},
 {'type': 8,
  'rate': 70.0,
  'max': 50,
  'open': False,
  'status

In [52]:
complete_race['results']

{'zeturfId': 252428,
 'status': 6,
 'positions': [{'position': 1, 'numbers': [4]},
  {'position': 2, 'numbers': [1]},
  {'position': 3, 'numbers': [5]},
  {'position': 4, 'numbers': [2]},
  {'position': 5, 'numbers': [6]}],
 'liveComments': []}

In [47]:
race['meetingId']

138

In [48]:
complete_race

{'get_url': 'https://www.unibet.fr/zones/turf/race.json?raceId=1003',
 'query_date': '2020-04-30T22:12:42.465135',
 'note': 'server error, no json'}

In [28]:
race.keys()

dict_keys(['zeturfId', 'meetingId', 'starttime', 'date', 'type', 'name', 'rank', 'conditions', 'stake', 'arjelLevel', 'distance', 'betCounter', 'friendlyUrl', 'runners', 'details', 'results'])

In [42]:
race['results']

{'zeturfId': 2109,
 'status': 6,
 'positions': [{'position': 1, 'numbers': [15]},
  {'position': 2, 'numbers': [4]},
  {'position': 3, 'numbers': [8]},
  {'position': 4, 'numbers': [6]},
  {'position': 5, 'numbers': [12]},
  {'position': 6, 'numbers': [14]},
  {'position': 7, 'numbers': [13]}],
 'liveComments': []}

In [41]:
race['betCounter']

In [34]:
print(race['meetingId'])

321

In [36]:
race['date']

'2005-08-22'

In [29]:
race['conditions']

"Départ à l'Autostart - Pour poulains entiers et hongres de 4 ans, n'ayant pas gagné 49.000 &euro; - Grande piste"

In [32]:
dt.datetime.fromtimestamp(race['starttime']/1000)

datetime.datetime(2005, 8, 22, 13, 50)

In [35]:
race['details']

{'zeturfId': 2109,
 'betList': [{'type': 1,
   'rate': 95.0,
   'max': 250,
   'open': False,
   'status': 16,
   'isolated': 0,
   'betstatus': 'PAID'},
  {'type': 2,
   'rate': 80.0,
   'max': 250,
   'open': False,
   'status': 16,
   'isolated': 0,
   'betstatus': 'PAID'},
  {'type': 3,
   'rate': 80.0,
   'max': 30,
   'open': False,
   'status': 16,
   'isolated': 0,
   'betstatus': 'PAID'},
  {'type': 5,
   'rate': 80.0,
   'max': 30,
   'open': False,
   'status': 16,
   'isolated': 0,
   'betstatus': 'PAID'},
  {'type': 6,
   'rate': 80.0,
   'max': 30,
   'open': False,
   'status': 16,
   'isolated': 0,
   'betstatus': 'PAID'}],
 'pronostic': None,
 'pronosticList': None,
 'raceDuration': '2\'33"97',
 'betState': 16,
 'photos': None,
 'raceBetStatus': 'PAID',
 'probables': None,
 'rates': None}

In [43]:
horse_show['rank']

1

In [44]:
race['rank']

1

In [48]:
with open(race_path, 'r') as fp:
    complete_race=json.load(fp=fp)

In [53]:
del complete_race['get_url'], complete_race['query_date']

In [51]:
race

{'zeturfId': 2109,
 'meetingId': 321,
 'starttime': 1124711400000,
 'date': '2005-08-22',
 'type': 'Attelé',
 'name': 'Prix de Jussey',
 'rank': 1,
 'conditions': "Départ à l'Autostart - Pour poulains entiers et hongres de 4 ans, n'ayant pas gagné 49.000 &euro; - Grande piste",
 'stake': 36000,
 'arjelLevel': '2',
 'distance': 2100,
 'betCounter': None,
 'friendlyUrl': '/turf/race/22-08-2005-R1-C1-vincennes-prix-de-jussey.html',
 'runners': [{'zeturfId': 27037,
   'raceId': 2109,
   'rank': 16,
   'name': 'NEVER DE LIVET (FR)',
   'jockey': 'Verbeeck J.',
   'weight': 0,
   'distance': 2100,
   'status': '40',
   'team': 0,
   'draw': 0,
   'blinkers': '0',
   'shoes': '',
   'position': '0',
   'silk': None,
   'betcounter': 0,
   'details': None},
  {'zeturfId': 27038,
   'raceId': 2109,
   'rank': 15,
   'name': 'NENUPHAR (FR)',
   'jockey': 'Levesque P.',
   'weight': 0,
   'distance': 2100,
   'status': '40',
   'team': 0,
   'draw': 0,
   'blinkers': '0',
   'shoes': '4',
   'pos

In [57]:
complete_race.keys()

dict_keys(['zeturfId', 'meetingId', 'starttime', 'date', 'type', 'name', 'rank', 'conditions', 'stake', 'arjelLevel', 'distance', 'betCounter', 'friendlyUrl', 'runners', 'details', 'results'])

In [58]:
race.keys()

dict_keys(['zeturfId', 'meetingId', 'starttime', 'date', 'type', 'name', 'rank', 'conditions', 'stake', 'arjelLevel', 'distance', 'betCounter', 'friendlyUrl', 'runners', 'details', 'results'])

In [60]:
shared_items = {k: complete_race[k] for k in complete_race if k in race and complete_race[k] == race[k]}
print(len(shared_items))

14


In [61]:
len(race)

16

In [63]:
difference_items = {k for k in complete_race if k in race and complete_race[k] != race[k]}

In [64]:
difference_items

{'details', 'runners'}

In [75]:
complete_race['runners']

[{'zeturfId': 27052,
  'raceId': 2109,
  'rank': 1,
  'name': 'NUMERO PRIVE (FR)',
  'jockey': 'Lannoo G.',
  'weight': 0,
  'distance': 2100,
  'status': '40',
  'team': 0,
  'draw': 0,
  'blinkers': '0',
  'shoes': '',
  'position': '0',
  'silk': None,
  'betcounter': 0,
  'details': {'zeturfId': 27052,
   'stakes': 41950,
   'musique': '1a 5a 3a 5a 1a 5a (04) 0a',
   'sex': 'M',
   'age': '6',
   'owner': 'M Bernard Marie Dubois',
   'trainer': 'Hojka B.',
   'coat': 'B.',
   'origins': "Cygnus d'Odyssee-Hollywood Melody",
   'comment': "Vient de remporter sa course à Enghien sur un parcours de vitesse. Se heurte à une opposition plus relevée. Pas hors d'affaire pour les places à l'issue d'un parcours caché.",
   'time': None,
   'reduction': None,
   'length': None}},
 {'zeturfId': 27051,
  'raceId': 2109,
  'rank': 2,
  'name': 'NICE MORNING (FR)',
  'jockey': 'Gilbert Martens',
  'weight': 0,
  'distance': 2100,
  'status': '40',
  'team': 0,
  'draw': 0,
  'blinkers': '0',
  's

In [69]:
race['runners'][0].keys()

dict_keys(['zeturfId', 'raceId', 'rank', 'name', 'jockey', 'weight', 'distance', 'status', 'team', 'draw', 'blinkers', 'shoes', 'position', 'silk', 'betcounter', 'details'])

In [70]:
x=complete_race['runners'][0]
y=race['runners'][0]
{k for k in x if k in y and x[k] != y[k]}

{'details', 'jockey', 'name', 'rank', 'zeturfId'}

In [78]:
race['runners'][0]

{'zeturfId': 27037,
 'raceId': 2109,
 'rank': 16,
 'name': 'NEVER DE LIVET (FR)',
 'jockey': 'Verbeeck J.',
 'weight': 0,
 'distance': 2100,
 'status': '40',
 'team': 0,
 'draw': 0,
 'blinkers': '0',
 'shoes': '',
 'position': '0',
 'silk': None,
 'betcounter': 0,
 'details': None}

In [76]:
for i, runner in enumerate(complete_race['runners']):
    print(i, runner['name'])

0 NUMERO PRIVE (FR)
1 NICE MORNING (FR)
2 NEW PISTOL (FR)
3 NEOPS RIVELLIERE (FR)
4 NICO D'AUSSY (FR)
5 NITBY (FR)
6 NEW LOOK DE CANISY (FR)
7 NIARPAN (FR)
8 NICCOLO D'AMOUR (FR)
9 NOVELINO (FR)
10 NESS MARCEAUX (FR)
11 NOBLE ANGEVIN (FR)
12 NEW HOPE (FR)
13 NO COMMENTARY (FR)
14 NENUPHAR (FR)
15 NEVER DE LIVET (FR)


In [79]:
complete_race['runners'][15]

{'zeturfId': 27037,
 'raceId': 2109,
 'rank': 16,
 'name': 'NEVER DE LIVET (FR)',
 'jockey': 'Verbeeck J.',
 'weight': 0,
 'distance': 2100,
 'status': '40',
 'team': 0,
 'draw': 0,
 'blinkers': '0',
 'shoes': '',
 'position': '0',
 'silk': None,
 'betcounter': 0,
 'details': {'zeturfId': 27037,
  'stakes': 35870,
  'musique': '5a 8a 2a Dm 3a (04) 2a Da',
  'sex': 'H',
  'age': '9',
  'owner': 'Mlle Caroline Cheradame',
  'trainer': 'Cheradame Mlle C.',
  'coat': 'Al.',
  'origins': 'Carpe Diem-vassia Pellois',
  'comment': "Possède un brin de qualité, mais risque encore de manquer  de compétition à ce niveau, venant juste d'effectuer sa rentrée sur l'herbe de Carentan. Malgré le talent de son driver, on se contentera de le regarder courir.",
  'time': None,
  'reduction': None,
  'length': None}}

In [81]:
complete_race['results']

{'zeturfId': 2109,
 'status': 6,
 'positions': [{'position': 1, 'numbers': [15]},
  {'position': 2, 'numbers': [4]},
  {'position': 3, 'numbers': [8]},
  {'position': 4, 'numbers': [6]},
  {'position': 5, 'numbers': [12]},
  {'position': 6, 'numbers': [14]},
  {'position': 7, 'numbers': [13]}],
 'liveComments': []}

In [82]:
for runner in complete_race['runners']:
    print(runner['rank'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
